In [1]:
from silq.meta_instruments.PulseSequence import PulseSequence
from silq.meta_instruments import pulses

In [2]:
pulse_sequence = PulseSequence()

In [3]:
sine_pulse1 = pulses.SinePulse(frequency=1e6, amplitude=1, t_start=0, t_stop=2)
sine_pulse2 = pulses.SinePulse(frequency=.4e6, amplitude=1, t_start=1, t_stop=3)

In [4]:
pulse_sequence.add_pulse(sine_pulse2)
pulse_sequence.add_pulse(sine_pulse1)
pulse_sequence.sort_pulses()

[SinePulse(f=1.00 MHz, A=1, t_start=0, t_stop=2),
 SinePulse(f=0.40 MHz, A=1, t_start=1, t_stop=3)]

In [41]:
x = [[],[],[]]

In [44]:
x[0] += [2]

In [47]:
x.sort()

In [29]:
class TestClass:
    def __init__(self, pulses):
        self.pulses = pulses
        
    def __getitem__(self, index):
        return self.pulses[index]
    
    def fun1(self):
        print('fun1')
        
    @staticmethod
    def fun2():
        print('fun2')

In [34]:
class TestClassChild(TestClass):
    def __init__(self, pulses):
        self.pulses = pulses
        
    def __getitem__(self, index):
        return self.pulses[index]
    
    def fun1(self):
        print('fun1')
        
    @staticmethod
    def fun2():
        print('fun2')

In [35]:
test_class = TestClassChild([1,2,3])

In [37]:
isinstance(test_class, TestClass)

True